In [2]:
# Importing Packages
import matplotlib.pyplot as plt
import random
import pandas as pd
from statistics import mean

In [3]:
# Creating Roll Dice Function
def trade(rr, win_rate, rpt, balance):
    outcome = random.choices([-1, 1 * rr], [1-win_rate, win_rate])[0] * rpt * balance
    return outcome

In [4]:
def get_dd(balance, max_trades = 100000):
    mySeries = pd.Series(balance) 

    Roll_Max = mySeries.rolling(max_trades, min_periods=1).max()
    Daily_Drawdown = mySeries - Roll_Max

    # Next we calculate the minimum (negative) daily drawdown in that window.
    # Again, use min_periods=1 if you want to allow the expanding window
    Max_Daily_Drawdown = Daily_Drawdown.rolling(max_trades, min_periods=1).min()

    # Plot the results
    # Daily_Drawdown.plot()
    # Max_Daily_Drawdown.plot()
    # mySeries.plot(secondary_y=True)
    return Max_Daily_Drawdown.iloc[-1]

In [6]:
def to_per(n):
    return round(n * 100, 2)

In [10]:
def simulator(trade_stats = [0.5,2,0.01, 30], funding_stats = [0.08, 0.12, 10000], num_simulations = 10000):
    win_rate, rr, rpt, max_trades = trade_stats
    target, max_dd, start_balance = funding_stats

#     print(f"""
# Simulation Parameters {num_simulations} Simulations::
# WIN_RATE: {to_per(win_rate)}% | RPT: {to_per(rpt)}% | RR: {rr} | #Trades_Per_Day: {trades_per_day}
# ---------------------------""")
          
    print(f"WIN_RATE: {to_per(win_rate)}% | RPT: {to_per(rpt)}% | RR: {rr} | #Trades: {max_trades}")
    win_probability = []
    drawdown = []
    end_balance = []
    funded = []

    # Creating Figure for Simulation Balances
    # fig = plt.figure()
    # plt.title("Monte Carlo Equity Simulator [" + str(num_simulations) + "simulations]")
    # plt.xlabel("Trade #")
    # plt.ylabel("Balance [$]")
    # plt.xlim([0, max_trades])
    
    # For loop to run for the number of simulations desired
    for i in range(num_simulations):
        balance = [start_balance]
        num_trades = [0]
        num_wins = 0

        while num_trades[-1] < max_trades:
            outcome = trade(rr, win_rate, rpt, start_balance)
            # Result if the dice are the same number
            balance.append(balance[-1] + outcome)
            num_wins += (1 if outcome > 0 else 0)
            num_trades.append(num_trades[-1] + 1)
    # Store tracking variables and add line to figure
        # dd = get_dd(balance, max_trades)
        dd = abs(start_balance - min(balance))
        drawdown.append(dd)
        win_probability.append(num_wins/num_trades[-1])
        end_balance.append(balance[-1])
        # print(balance[-1], 10000 * (1+target), abs(dd), start_balance * max_dd)
        if (balance[-1] > 10000 * (1+target)) and (abs(dd) < start_balance * max_dd):
            funded.append(1)
        else:
            funded.append(0)
        # plt.plot(num_trades, balance)
# Showing the plot after the simulations are finished
    # plt.show()

    # Averaging win probability and end balance
    # overall_win_probability = sum(win_probability)/len(win_probability)
    overall_end_balance = mean(end_balance)
    overall_funding_probability = mean(funded)
    # Displaying the averages
    # print("Average win probability after " + str(num_simulations) + " runs: " + str(overall_win_probability))
    biggest_max_dd = abs(max(drawdown))
    avg_max_dd = abs(mean(drawdown))
    print(f"Average funding probability: {to_per(overall_funding_probability)}%")
    return [to_per(overall_funding_probability), biggest_max_dd, avg_max_dd]
    # print(f"Average Performance: ${overall_end_balance} ({to_per(overall_end_balance/start_balance - 1)}%)")
    # print(f"Biggest Max DD: ${biggest_max_dd} ({to_per(biggest_max_dd/start_balance)}%)")
    # print(f"Average Max DD: ${avg_max_dd} ({to_per(avg_max_dd/start_balance)}%)")


In [11]:
if __name__ == "__main__":
    df = pd.DataFrame(columns=['Win%', 'RPT', 'RR', 'Trades', 'FundingProbability', 'Max_DD', 'Avg_DD'])
    rpt_list = [0.0025,0.005,0.01,0.02]
    num_trade_list = [15,30,60,90,120]
    for i in rpt_list:
        for j in num_trade_list:
            df.loc[len(df.index)] = [0.5, i, 2, j] + simulator(trade_stats = [0.5,2,i,j])
            print("=======================")

WIN_RATE: 50.0% | RPT: 0.25% | RR: 2 | #Trades: 15
Average funding probability: 0%
WIN_RATE: 50.0% | RPT: 0.25% | RR: 2 | #Trades: 30
Average funding probability: 2.25%
WIN_RATE: 50.0% | RPT: 0.25% | RR: 2 | #Trades: 60
Average funding probability: 45.25%
WIN_RATE: 50.0% | RPT: 0.25% | RR: 2 | #Trades: 90
Average funding probability: 82.33%
WIN_RATE: 50.0% | RPT: 0.25% | RR: 2 | #Trades: 120
Average funding probability: 95.86%
WIN_RATE: 50.0% | RPT: 0.5% | RR: 2 | #Trades: 15
Average funding probability: 6.27%
WIN_RATE: 50.0% | RPT: 0.5% | RR: 2 | #Trades: 30
Average funding probability: 42.41%
WIN_RATE: 50.0% | RPT: 0.5% | RR: 2 | #Trades: 60
Average funding probability: 88.24%
WIN_RATE: 50.0% | RPT: 0.5% | RR: 2 | #Trades: 90
Average funding probability: 97.65%
WIN_RATE: 50.0% | RPT: 0.5% | RR: 2 | #Trades: 120
Average funding probability: 99.72%
WIN_RATE: 50.0% | RPT: 1.0% | RR: 2 | #Trades: 15
Average funding probability: 50.3%
WIN_RATE: 50.0% | RPT: 1.0% | RR: 2 | #Trades: 30
Aver

In [12]:
df.to_csv('temp')

In [85]:
import os
os.getcwd()

'/Users/yash/Desktop/Trading/code/ep/src'